In [1]:
###################################################################################################
#
# resolution_comparison_plot.py 	(c) Ian Williams, Adnan Khan, Matt McQuinn
#     				    	        ianw89@live.com
#
#  This produces the plots in Appendix C of Khan, Williams, McQuinn 2022
#
###################################################################################################

import matplotlib.pyplot as plt
from cgmbrush.cgmbrush import *
from cgmbrush.plots.plots import *
import matplotlib.ticker as mticker
import plotting_routines as makefig

In [2]:
date = '2022-04-04'
provider = BolshoiProvider()

### PLOT SETTINGS ###
axis_fontsize = 12
curve_lw = 3
axis_label = 24
sub_title_size= 22

plt.rc('axes', titlesize=axis_fontsize)     # fontsize of the axes title
plt.rc('axes', labelsize=axis_label)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=XBIG_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=XBIG_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=18)    # legend fontsize
plt.rc('figure', titlesize=sub_title_size)  # fontsize of the figure title

In [ ]:

STH_config = Configuration(SphericalTophatProfile(), 1, provider=provider, resolution=1)
STH_config.datestamp = date
STH1_256 = STH_config.get_addition_field()
STH1_hist = create_histograms(STH_config.get_final_field(), STH_config.resolution*1024)
STH_config.clear_results()

STH_config = Configuration(SphericalTophatProfile(), 1, provider=provider, resolution=2)
STH_config.datestamp = date
STH2_256 = STH_config.get_addition_field()
STH2_hist = create_histograms(STH_config.get_final_field(), STH_config.resolution*1024)
STH_config.clear_results()

STH_config = Configuration(SphericalTophatProfile(), 1, provider=provider, resolution=4)
STH_config.datestamp = date
STH4_256 = STH_config.get_addition_field()
STH4_hist = create_histograms(STH_config.get_final_field(), STH_config.resolution*1024)
STH_config.clear_results()

STH_config = Configuration(SphericalTophatProfile(), 1, provider=provider, resolution=8)
STH_config.datestamp = date
STH8_256 = STH_config.get_addition_field()
STH8_hist = create_histograms(STH_config.get_final_field(), STH_config.resolution*1024)
STH_config.clear_results()

STH_config = Configuration(SphericalTophatProfile(), 1, provider=provider, resolution=16)
STH_config.datestamp = date
STH16_256 = STH_config.get_addition_field()
STH16_hist = create_histograms(STH_config.get_final_field(), STH_config.resolution*1024)
STH_config.clear_results()

STH_config = Configuration(SphericalTophatProfile(), 1, provider=provider, resolution=32)
STH_config.datestamp = date
STH32_256 = STH_config.get_addition_field()
STH32_hist = create_histograms(STH_config.get_final_field(), STH_config.resolution*1024)
STH_config.clear_results()

fire_config = Configuration(FireProfile(), 1, provider=provider, resolution=1)
fire_config.datestamp = date
fire1_hist = create_histograms(fire_config.get_final_field(), fire_config.resolution*1024)
fire_config.clear_results()

fire_config = Configuration(FireProfile(), 1, provider=provider, resolution=2)
fire_config.datestamp = date
fire2_hist = create_histograms(fire_config.get_final_field(), fire_config.resolution*1024)
fire_config.clear_results()

fire_config = Configuration(FireProfile(), 1, provider=provider, resolution=4)
fire_config.datestamp = date
fire4_hist = create_histograms(fire_config.get_final_field(), fire_config.resolution*1024)
fire_config.clear_results()

fire_config = Configuration(FireProfile(), 1, provider=provider, resolution=8)
fire_config.datestamp = date
fire8_hist = create_histograms(fire_config.get_final_field(), fire_config.resolution*1024)
fire_config.clear_results()

fire_config = Configuration(FireProfile(), 1, provider=provider, resolution=16)
fire_config.datestamp = date
fire16_hist = create_histograms(fire_config.get_final_field(), fire_config.resolution*1024)
fire_config.clear_results()

fire_config = Configuration(FireProfile(), 1, provider=provider, resolution=32)
fire_config.datestamp = date
fire32_hist = create_histograms(fire_config.get_final_field(), fire_config.resolution*1024)
fire_config.clear_results()



In [ ]:

def make_resolution_image_comparison(vmax):
    """Image plots of methodology. Comparing resolutions of the halos re-added density field."""

    u=15
    vmin = 0

    fields = [
        ('Resolution: 1024', STH1_256[0,0:1*u,0:1*u]),
        ('Resolution: 2048', STH2_256[0,0:2*u,0:2*u]),
        #('Resolution: 4096', STH4_256[0,0:4*u,0:4*u]),
        ('Resolution: 8192', STH8_256[0,0:8*u,0:8*u]),
        #('Resolution: 16384', STH16_256[0,0:16*u,0:16*u]),
        ('Resolution: 32768', STH32_256[0,0:32*u,0:32*u])
    ]
    
    fig, axs = fields_comparison_plot(fields, vmin, vmax)
    saveFig('implot_resolution_comparison_%s.pdf' % vmax, fig, bbox_inches='tight')

 

def make_resolution_DM_histogram_comparison():

    fig, axs = plt.subplots(1, 2, sharex='col', sharey='row',
                            gridspec_kw={'hspace': 0, 'wspace': 0},figsize=(20,10))

    # fig, axs = plt.subplots(2, 1, sharex='col', sharey='row',
    #                         gridspec_kw={'hspace': 0, 'wspace': 0},figsize=(10,20))


    # Left panel
    axs[0].plot(STH1_hist[0],STH1_hist[1] / np.sum(STH1_hist[1]),'-',label='1024',lw=curve_lw,color='r')
    axs[0].plot(STH2_hist[0],STH2_hist[1] / np.sum(STH2_hist[1]),'-',label='2048',lw=curve_lw,color='g')
    axs[0].plot(STH4_hist[0],STH4_hist[1] / np.sum(STH4_hist[1]),'-',label='4096',lw=curve_lw,color='b')
    axs[0].plot(STH8_hist[0],STH8_hist[1] / np.sum(STH8_hist[1]),'-',label='8192',lw=curve_lw,color='k')
    axs[0].plot(STH16_hist[0],STH16_hist[1] / np.sum(STH16_hist[1]),'-',label='16384',lw=curve_lw,color='m')
    axs[0].plot(STH32_hist[0],STH32_hist[1] / np.sum(STH32_hist[1]),'-',label='32768',lw=curve_lw,color='c')

    # Right panel
    axs[1].plot(fire1_hist[0],fire1_hist[1] / np.sum(fire1_hist[1]),'-',label='1024',lw=curve_lw,color='r')
    axs[1].plot(fire2_hist[0],fire2_hist[1] / np.sum(fire2_hist[1]),'-',label='2048',lw=curve_lw,color='g')
    axs[1].plot(fire4_hist[0],fire4_hist[1] / np.sum(fire4_hist[1]),'-',label='4096',lw=curve_lw,color='b')
    axs[1].plot(fire8_hist[0],fire8_hist[1] / np.sum(fire8_hist[1]),'-',label='8192',lw=curve_lw,color='k')
    axs[1].plot(fire16_hist[0],fire16_hist[1] / np.sum(fire16_hist[1]),'-',label='16384',lw=curve_lw,color='m')
    axs[1].plot(fire32_hist[0],fire32_hist[1] / np.sum(fire32_hist[1]),'-',label='32768',lw=curve_lw,color='c')

    axs[0].set_xlim(0,200)
    axs[1].set_xlim(0.01,200)

    axs[0].set_ylabel('P(DM)',fontsize='30')

    axs[0].legend(loc='upper right',prop={'size':'25'})
    # plt.setp(leg.get_title(),fontsize=20)

    # plt.ylim(0,1)

    fig.text(0.5, 0.04, 'DM [pc cm$^{-3}$]', ha='center',fontsize=30)

    axs[0].set_title('Profile: Spherical Tophat',size='30')
    axs[1].set_title('Profile: FIRE ',size='30')

    # Size of ticks
    axs[0].tick_params(axis='both', which='major', labelsize=24)
    axs[1].tick_params(axis='both', which='major', labelsize=24)

    saveFig('hist_resolution_comparison.pdf', fig, bbox_inches='tight')

In [ ]:
#make_resolution_image_comparison(500)
make_resolution_image_comparison(600)
#make_resolution_image_comparison(750)
#make_resolution_image_comparison(1000)

make_resolution_DM_histogram_comparison()

In [3]:

STH_config_256 = Configuration(SphericalTophatProfile(), 1, provider=provider, resolution=32)
STH_config_256.datestamp = date
STH_config_512 = Configuration(SphericalTophatProfile(), 1, provider=provider, resolution=32, den_grid_size=512)
STH_config_512.datestamp = date

FIRE_config_256 = Configuration(FireProfile(), 1, provider=provider, resolution=32)
FIRE_config_256.datestamp = date
FIRE_config_512 = Configuration(FireProfile(), 1, provider=provider, resolution=32, den_grid_size=512)
FIRE_config_512.datestamp = date

NFW_config_256 = Configuration(NFWProfile(), 1, provider=provider, resolution=32)
NFW_config_256.datestamp = date
NFW_config_512 = Configuration(NFWProfile(), 1, provider=provider, resolution=32, den_grid_size=512)
NFW_config_512.datestamp = date

In [ ]:
# Histograms comparing 256 and 512 original grid for both raw and addition fields

def res_compare_256_512(config256, config512, name, resolution):

    fig_hist = plt.figure(figsize=(20,10))

    original_field_256 = provider.get_density_field(0, 256)
    original_field_512 = provider.get_density_field(0, 512)

    hist_256 = histArray(original_field_256,100,256,0,300)
    hist_512 = histArray(original_field_512,200,512,0,300)

    hist_256_rem = histArray(config256.get_final_field()[0,:,:],100,256*4*resolution,0,300)
    hist_512_rem = histArray(config512.get_final_field()[0,:,:],200,512*2*resolution,0,300)

    plt.plot(hist_256[0],hist_256[1],'-',label='256 Raw density field',lw=curve_lw,color='r')
    plt.plot(hist_512[0],hist_512[1],'-',label='512 Raw density field',lw=curve_lw,color='b')

    plt.plot(hist_256_rem[0],hist_256_rem[1],'--',label='256 Final density field',lw=curve_lw,color='r')        
    plt.plot(hist_512_rem[0],hist_512_rem[1],'--',label='512 Final density field',lw=curve_lw,color='b')

    plt.xlabel('DM [pc cm$^{-3}$]',fontsize='30')
    plt.ylabel('P(DM)',fontsize='30')
    plt.xlim(0,250)
    plt.legend(loc='upper right',prop={'size':'25'})
    plt.ylim(0,.021)

    # Something is broken with this for me TODO
    #f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
    #g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.1e' % x))
    #plt.gca().yaxis.set_major_formatter(mticker.FuncFormatter(g))

    saveFig('hist_add_remove_halos_{}.pdf'.format(name), fig_hist, bbox_inches='tight')

res_compare_256_512(STH_config_256, STH_config_512, 'STH', 32)
#res_compare_256_512(FIRE_config_256, FIRE_config_512, 'FIRE', 8)


In [6]:
# DM vs R Compare 512 and 256 res

xmin = 25
xmax = 2399

M_chosen = [10,22,34] 

# They are the same to 9 digits of precision; doesn't affect results
#print(np.mean(provider.get_density_field(0, 256)))
#print(np.mean(provider.get_density_field(0, 512)))

# Compare STH
ymax = [99,149,249]
series = [ 
    (STH_config_256.generate_DM_vs_radius_profile(load_from_files=True),STH_config_256.generate_profile_of_masks(load_from_files=True),'3D Tophat 256','r','-'), 
    (STH_config_512.generate_DM_vs_radius_profile(load_from_files=True),STH_config_512.generate_profile_of_masks(load_from_files=True),'3D Tophat 512','b','-') 
    ]
makefig.make_DM_vs_Rad_profiles_plots(series, False, True, xmin, xmax, STH_config_256.resolution, STH_config_256.resolution*provider.halofieldresolution, M_chosen, STH_config_512.get_virial_radii(), provider, STH_config_512.get_halo_masses(), ymax, 'STH_256_vs_512')

# Compare Fire
ymax = [129,309,799] 
series = [ 
    (FIRE_config_256.generate_DM_vs_radius_profile(load_from_files=True),FIRE_config_256.generate_profile_of_masks(load_from_files=True),'Fire 256','r','-'), 
    (FIRE_config_512.generate_DM_vs_radius_profile(load_from_files=True),FIRE_config_512.generate_profile_of_masks(load_from_files=True),'Fire 512','b','-') 
    ]
makefig.make_DM_vs_Rad_profiles_plots(series, False, True, xmin, xmax, FIRE_config_256.resolution, FIRE_config_256.resolution*provider.halofieldresolution, M_chosen, STH_config_512.get_virial_radii(), provider, STH_config_512.get_halo_masses(), ymax, 'FIRE_256_vs_512')

# Make the version we use in the paper
M_chosen = [22] 
ymax = [309] 
series = [ 
    (STH_config_256.generate_DM_vs_radius_profile(load_from_files=True),STH_config_256.generate_profile_of_masks(load_from_files=True),'3D Tophat 256','r','-'), 
    (STH_config_512.generate_DM_vs_radius_profile(load_from_files=True),STH_config_512.generate_profile_of_masks(load_from_files=True),'3D Tophat 512','b','-'),
    (NFW_config_256.generate_DM_vs_radius_profile(load_from_files=True),NFW_config_256.generate_profile_of_masks(load_from_files=True),'NFW 256','r','--'), 
    (NFW_config_512.generate_DM_vs_radius_profile(load_from_files=True),NFW_config_512.generate_profile_of_masks(load_from_files=True),'NFW 512','b','--') 
    ]
makefig.make_DM_vs_Rad_profiles_plots(series, False, False, xmin, xmax, NFW_config_256.resolution, NFW_config_256.resolution*provider.halofieldresolution, M_chosen, STH_config_512.get_virial_radii(), provider, STH_config_512.get_halo_masses(), ymax, 'Paper_256_vs_512')

# Make a plot of the difference between 256 and 512
M_chosen = [10,22,34] 
mean_DM = np.mean(provider.get_density_field(0, 256)) # this gets subtracted off later we need to 'undo' that for the difference version of this
print(np.shape(STH_config_512.DM_vs_R1))
delta1 = (STH_config_512.DM_vs_R1 - STH_config_256.DM_vs_R1) + mean_DM
delta2 = (NFW_config_512.DM_vs_R1 - NFW_config_256.DM_vs_R1) + mean_DM

ymax = [49,49,49]
series = [ 
    (delta1,[],'3D Tophat $\Delta$','r','-'), 
    (delta2,[],'NFW $\Delta$','r','-'), 
    ]
makefig.make_DM_vs_Rad_profiles_plots(series, False, False, xmin, xmax, NFW_config_256.resolution, NFW_config_256.resolution*provider.halofieldresolution, M_chosen, STH_config_512.get_virial_radii(), provider, STH_config_512.get_halo_masses(), ymax, 'Delta_256_vs_512', y_label='$\Delta$ DM [pc cm$^{-3}$]')


saving output to  /Volumes/Seagate Backup Plus Drive/CGM-FRB-Data/DMvsRad_STH_256_vs_512_32_2399.pdf
(60, 227)
saving output to  /Volumes/Seagate Backup Plus Drive/CGM-FRB-Data/DMvsRad_Delta_256_vs_512_32_2399.pdf
